We start from an Imagenet-pretrained ResNet152, replace its first and last layers, make it ready to use multi-resolution inputs generated from a single image, and finetune on CIFAR-100.

We then train separate linear layers for each of the intermediate representations on top of a frozen backbone model.

From these, we form a self-ensemble. We evalaute its adversarial accuracy on CIFAR-100 using the RobustBench AutoAttack at the end (with the `rand` flag enable).

The whole Colab should take ~60 minutes on an A100 GPU and should be self-contained.

It should give you above/about SOTA adversarial robustness on CIFAR-100 under $L_\infty = 8/255$ attacks already, visualize the successfully attacked images and also visualize the class prototypes optimized directly from pixels.

In [1]:
!nvidia-smi
!pip install torch torchvision numpy tqdm matplotlib safetensors

zsh:1: command not found: nvidia-smi


In [ ]:
from torch.amp import autocast, GradScaler
from torchvision.models import resnet152, ResNet152_Weights
from types import SimpleNamespace
import json
from safetensors import save_file
import copy
import hashlib
import os
import random
import time
from contextlib import contextmanager
from pathlib import Path
from torchvision.models import resnet152, ResNet152_Weights
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import datasets, models
from torchvision.models import resnet152
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import random
import hashlib
import time
import copy
import matplotlib.pyplot as plt
from contextlib import contextmanager
from tqdm import tqdm
import gc

assert torch.cuda.is_available()

#
# config
#


classes_path = Path.cwd().parent / "data"
dataset_path = Path.cwd().parent / "datasets"
weights_path = Path.cwd().parent / "weights"

os.makedirs(classes_path, exist_ok=True)
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(weights_path, exist_ok=True)


#
# perf
#


def free_mem():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    torch.cuda.ipc_collect()


free_mem()
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision("high")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    "backend:native,",
    "max_split_size_mb:512,",
    "garbage_collection_threshold:0.8,",
    "expandable_segments:True",
)

# seed = 41
# random.seed(seed)
# np.random.seed(seed)
# torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True


@contextmanager
def isolated_environment():
    # save and restore random states to seperate the random-seed-fixing behavior from attacks
    np_random_state = np.random.get_state()
    python_random_state = random.getstate()
    torch_random_state = torch.get_rng_state()
    cuda_random_state = torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None
    numpy_print_options = np.get_printoptions()
    try:
        yield  # execute code block
    finally:
        np.random.set_state(np_random_state)
        random.setstate(python_random_state)
        torch.set_rng_state(torch_random_state)
        if cuda_random_state:
            torch.cuda.set_rng_state_all(cuda_random_state)
        np.set_printoptions(**numpy_print_options)


#
# data
#


classes = 100

if classes == 10:
    trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
    testset = datasets.CIFAR10(root=dataset_path, train=False, download=True)
    original_images_train_np = np.array(trainset.data)
    original_labels_train_np = np.array(trainset.targets)
    original_images_test_np = np.array(testset.data)
    original_labels_test_np = np.array(testset.targets)
    class_names_cifar10 = json.loads((classes_path / "cifar10_classes.json").read_text())
elif classes == 100:
    trainset = datasets.CIFAR100(root=dataset_path, train=True, download=True)
    testset = datasets.CIFAR100(root=dataset_path, train=False, download=True)
    original_images_train_np = np.array(trainset.data)
    original_labels_train_np = np.array(trainset.targets)
    original_images_test_np = np.array(testset.data)
    original_labels_test_np = np.array(testset.targets)
    class_names_cifar100 = json.loads((classes_path / "cifar100_classes.json").read_text())
else:
    assert False

images_train_np = original_images_train_np / 255.0  # scale to [0, 1]
images_test_np = original_images_test_np / 255.0
labels_train_np = original_labels_train_np
labels_test_np = original_labels_test_np


#
# multi resolution preprocessing (channel layer)
#


def custom_rand(input_tensor, size):
    return torch.Tensor(np.random.rand(*size)).to("cuda")


def custom_choices(items, tensor):
    return np.random.choice(items, (len(tensor)))


resolutions = [32, 16, 8, 4]  # pretty arbitrary
shuffle_image_versions_randomly = False  # to shuffle randomly which image is which in the multi-res stack (false in paper)
transform = True  # to apply the transformations or not (true in paper)


def default_make_multichannel_input(images):
    return torch.concatenate([images] * len(resolutions), axis=1)


def apply_transformations(images, down_res, up_res, jit_x, jit_y, down_noise, up_noise, contrast, color_amount):
    # images = torch.mean(images,axis=1,keepdims=True) # for MNIST alone

    images_collected = []
    for i in range(images.shape[0]):
        image = images[i]
        image = torchvision.transforms.functional.adjust_contrast(image, contrast[i])  # changing contrast
        image = torch.roll(image, shifts=(jit_x[i], jit_y[i]), dims=(-2, -1))  # shift the result in x and y
        image = color_amount[i] * image + torch.mean(image, axis=0, keepdims=True) * (1 - color_amount[i])  # shifting in the color <-> grayscale axis
        images_collected.append(image)

    images = torch.stack(images_collected, axis=0)

    images = F.interpolate(images, size=(down_res, down_res), mode="bicubic")  # descrease the resolution
    noise = down_noise * custom_rand(images + 312, (images.shape[0], 3, down_res, down_res)).to("cuda")  # low res noise
    images = images + noise

    images = F.interpolate(images, size=(up_res, up_res), mode="bicubic")  # increase the resolution
    noise = up_noise * custom_rand(images + 812, (images.shape[0], 3, up_res, up_res)).to("cuda")  # high res noise
    images = images + noise

    images = torch.clip(images, 0, 1)  # clipping to the right range of values
    return images


def make_multichannel_input(images):
    all_channels = []
    if transform:
        for i, r in enumerate(resolutions):
            jit_size = 3  # max size of the x-y jit in each axis, sampled uniformly from -jit_size to +jit_size inclusive
            images_now = apply_transformations(
                images,
                down_res=r,
                up_res=32,  # hard coded for CIFAR-10 or CIFAR-100
                jit_x=custom_choices(range(-jit_size, jit_size + 1), images + i),  # x-shift
                jit_y=custom_choices(range(-jit_size, jit_size + 1), 51 * images + 7 * i + 125 * r),  # y-shift
                down_noise=0.2,  # noise standard deviation to be added at the low resolution
                up_noise=0.2,  # noise stadard deviation to be added at the high resolution
                contrast=custom_choices(np.linspace(0.5, 1.0, 100), 5 + 7 * images + 8 * i + 2 * r),  # change in contrast
                color_amount=custom_choices(np.linspace(0.5, 1.0, 100), 5 + 7 * images + 8 * i + 2 * r),  # change in color amount
            )
            all_channels.append(images_now)
    else:
        all_channels = [images] * len(resolutions)

    if not shuffle_image_versions_randomly:
        return torch.concatenate(all_channels, axis=1)
    elif shuffle_image_versions_randomly:
        indices = torch.randperm(len(all_channels))
        shuffled_tensor_list = [all_channels[i] for i in indices]
        return torch.concatenate(shuffled_tensor_list, axis=1)


demo = False
if demo:
    sample_images = images_test_np[:5]
    for j in [0, 1]:
        multichannel_images = make_multichannel_input(torch.Tensor(sample_images.transpose([0, 3, 1, 2])).to("cuda")).detach().cpu().numpy().transpose([0, 2, 3, 1])
        N = 1 + multichannel_images.shape[3] // 3
        plt.figure(figsize=(N * 5.5, 5))
        plt.subplot(1, N, 1)
        plt.title("original")
        plt.imshow(sample_images[j])
        plt.xticks([], [])
        plt.yticks([], [])
        for i in range(N - 1):
            plt.subplot(1, N, i + 2)
            plt.title(f"res={resolutions[i]}")
            plt.imshow(multichannel_images[j, :, :, 3 * i : 3 * (i + 1)])
            plt.xticks([], [])
            plt.yticks([], [])
        plt.show()


#
# backbone training
#


def eval_model(model, images_in, labels_in, batch_size=128):
    all_preds = []
    all_logits = []

    with torch.no_grad():
        its = int(np.ceil(float(len(images_in)) / float(batch_size)))

        pbar = tqdm(range(its), desc="eval", ncols=100)

        for it in pbar:
            i1 = it * batch_size
            i2 = min([(it + 1) * batch_size, len(images_in)])

            inputs = torch.Tensor(images_in[i1:i2].transpose([0, 3, 1, 2])).to("cuda")
            outputs = model(inputs)

            all_logits.append(outputs.detach().cpu().numpy())

            preds = torch.argmax(outputs, axis=-1)
            all_preds.append(preds.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_logits = np.concatenate(all_logits, axis=0)

    return np.sum(all_preds == labels_in), all_preds.shape[0], all_logits


def fgsm_attack(model, xs, ys, epsilon, random_reps=1, batch_size=64):
    model = model.eval()

    its = int(np.ceil(xs.shape[0] / batch_size))

    all_perturbed_images = []

    for it in range(its):
        i1 = it * batch_size
        i2 = min([(it + 1) * batch_size, xs.shape[0]])

        x = torch.Tensor(xs[i1:i2].transpose([0, 3, 1, 2])).to("cuda")
        y = torch.Tensor(ys[i1:i2]).to("cuda").to(torch.long)

        x.requires_grad = True

        for _ in range(random_reps):
            outputs = model(x)
            loss = nn.CrossEntropyLoss()(outputs, y)
            loss.backward()

        perturbed_image = x + epsilon * x.grad.data.sign()
        perturbed_image = torch.clip(perturbed_image, 0, 1)

        all_perturbed_images.append(perturbed_image.detach().cpu().numpy().transpose([0, 2, 3, 1]))

    return np.concatenate(all_perturbed_images, axis=0)


def train_model(model_in, images_in, labels_in, epochs, lr, batch_size, optimizer_in, subset_only, mode, use_adversarial_training, adversarial_epsilon, skip_test_set_eval):
    global storing_models
    if mode == "train":
        model_in.train()
    elif mode == "eval":
        model_in.eval()
    else:
        assert False

    if subset_only is None:
        train_optimizer = optimizer_in(model_in.parameters(), lr=lr)
    else:
        train_optimizer = optimizer_in(subset_only, lr=lr)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        print(f"epoch {epoch+1}/{epochs}")
        if mode == "train":
            model_in.train()
        elif mode == "eval":
            model_in.eval()

        all_hits = []
        randomized_ids = np.random.permutation(range(len(images_in)))

        its = int(np.ceil(float(len(images_in)) / float(batch_size)))
        pbar = tqdm(range(its), desc="Training", ncols=100)
        for it in pbar:
            i1 = it * batch_size
            i2 = min([(it + 1) * batch_size, len(images_in)])

            np_images_used = images_in[randomized_ids[i1:i2]]
            np_labels_used = labels_in[randomized_ids[i1:i2]]

            # very light adversarial training if on
            if use_adversarial_training:
                attacked_images = fgsm_attack(model_in.eval(), np_images_used[:], np_labels_used[:], epsilon=adversarial_epsilon, random_reps=1, batch_size=batch_size // 2)
                np_images_used = attacked_images
                np_labels_used = np_labels_used
                if mode == "train":
                    model_in.train()
                elif mode == "eval":
                    model_in.eval()

            inputs = torch.Tensor(np_images_used.transpose([0, 3, 1, 2])).to("cuda")
            labels = torch.Tensor(np_labels_used).to("cuda").to(torch.long)

            # the actual optimization step
            train_optimizer.zero_grad()
            outputs = model_in(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            train_optimizer.step()

            # on the fly eval for the train set batches
            preds = torch.argmax(outputs, axis=-1)
            acc = torch.mean((preds == labels).to(torch.float), axis=-1)
            all_hits.append((preds == labels).to(torch.float).detach().cpu().numpy())
            train_accs.append(acc.detach().cpu().numpy())
            pbar.set_description(f"train acc={acc.detach().cpu().numpy()} loss={loss.item()}")

        # end of epoch eval
        if not skip_test_set_eval:
            with isolated_environment():
                eval_model_copy = copy.deepcopy(model_in)
                test_hits, test_count, _ = eval_model(eval_model_copy.eval(), images_test_np, labels_test_np)
        train_hits = np.sum(np.concatenate(all_hits, axis=0).reshape([-1]))
        train_count = np.concatenate(all_hits, axis=0).reshape([-1]).shape[0]
        print(f"e={epoch} train {train_hits} / {train_count} = {train_hits/train_count},  test {test_hits} / {test_count} = {test_hits/test_count}")
        test_accs.append((test_hits / test_count) if ((not skip_test_set_eval) and (test_count > 0)) else 0)

    print("done")
    return model_in


imported_model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)

in_planes = 3
planes = 64
stride = 2
N = len(resolutions)  # input channels multiplier due to multi-res input
conv2 = nn.Conv2d(N * in_planes, planes, kernel_size=7, stride=stride, padding=3, bias=False)
imported_model.conv1 = copy.deepcopy(conv2)  # replace pretrained conv with multi-res one


class ImportedModelWrapper(nn.Module):
    def __init__(self, imported_model, multichannel_fn):
        super(ImportedModelWrapper, self).__init__()
        self.imported_model = imported_model
        self.multichannel_fn = multichannel_fn

    def forward(self, x):
        # custom preprocessing
        x = self.multichannel_fn(x)
        # resnet default preprocessing
        x = F.interpolate(x, size=(224, 224), mode="bicubic")
        x = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406] * (x.shape[1] // 3), std=[0.229, 0.224, 0.225] * (x.shape[1] // 3))(x)
        x = self.imported_model(x)
        return x


free_mem()
wrapped_model = ImportedModelWrapper(imported_model, make_multichannel_input).to("cuda")
wrapped_model.multichannel_fn = make_multichannel_input
model = copy.deepcopy(wrapped_model)
model.multichannel_fn = make_multichannel_input
model = model.train()

train_accs = []
test_accs = []

free_mem()
# scaler = GradScaler("cuda") # scaler and compiler are not that helpful
with torch.autocast("cuda"):
    model = train_model(
        model_in=model,
        images_in=images_train_np,
        labels_in=labels_train_np,
        epochs=6,  # 2h on A100 gpu
        lr=3.3e-5,  # probably not optimal
        batch_size=128,  # reduce if you have mem issues
        optimizer_in=optim.Adam,
        subset_only=None,
        mode="train",
        use_adversarial_training=False,
        adversarial_epsilon=8 / 255,
        skip_test_set_eval=False,
    )

In [2]:
torch.set_float32_matmul_precision("high")  # delete from here
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (  # delete from here
    "backend:native,",
    "max_split_size_mb:512,",
    "garbage_collection_threshold:0.8,",
    "expandable_segments:True",
)


def free_mem():  # delete from here
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    torch.cuda.ipc_collect()


#
# ensembled training
#


class WrapModelForResNet152(torch.nn.Module):
    def __init__(self, model, multichannel_fn, classes=10):
        super(WrapModelForResNet152, self).__init__()
        self.multichannel_fn = multichannel_fn
        self.model = model
        self.classes = classes
        self.layer_operations = [
            torch.nn.Sequential(
                model.conv1,
                model.bn1,
                model.relu,
                model.maxpool,
            ),
            *model.layer1,
            *model.layer2,
            *model.layer3,
            *model.layer4,
            model.avgpool,
            model.fc,
        ]
        self.all_dims = [
            3 * 224 * 224 * len(resolutions),
            64 * 56 * 56,
            *[256 * 56 * 56] * len(model.layer1),
            *[512 * 28 * 28] * len(model.layer2),
            *[1024 * 14 * 14] * len(model.layer3),
            *[2048 * 7 * 7] * len(model.layer4),
            2048,
            1000,
        ]

        class BatchNormLinear(nn.Module):
            def __init__(self, in_features, out_features, device="cuda"):
                super(BatchNormLinear, self).__init__()
                self.batch_norm = nn.BatchNorm1d(in_features, device="cuda")  # batch norm
                self.linear = nn.Linear(in_features, out_features, device="cuda")  # linear probe

            def forward(self, x):
                x = self.batch_norm(x)
                return self.linear(x)

        self.linear_layers = torch.nn.ModuleList([BatchNormLinear(self.all_dims[i], classes, device="cuda") for i in range(len(self.all_dims))])

    def prepare_input(self, x):  # preprocessing
        # custom preprocessing
        x = self.multichannel_fn(x)
        # resnet default preprocessing
        x = F.interpolate(x, size=(224, 224), mode="bicubic")
        x = torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406] * (x.shape[1] // 3), std=[0.229, 0.224, 0.225] * (x.shape[1] // 3))(x)
        return x

    def forward_until(self, x, layer_id):  # forward until layer
        x = self.prepare_input(x)

        for l in range(layer_id):
            if list(x.shape)[1:] == [2048, 1, 1]:
                x = x.reshape([-1, 2048])

            x = self.layer_operations[l](x)
        return x

    def forward_after(self, x, layer_id):  # forward after layer
        x = self.prepare_input(x)
        for l in range(layer_id, len(self.layer_operations)):
            if list(x.shape)[1:] == [2048, 1, 1]:
                x = x.reshape([-1, 2048])

            x = self.layer_operations[l](x)
        return x

    def predict_from_layer(self, x, l):  # predict in layer
        x = self.forward_until(x, l)
        x = x.reshape([x.shape[0], -1])
        return self.linear_layers[l](x)

    def predict_from_several_layers(self, x, layers):  # predict in several layers
        x = self.prepare_input(x)
        outputs = dict()
        outputs[0] = self.linear_layers[0](x.reshape([x.shape[0], -1]))
        for l in range(len(self.layer_operations)):
            if list(x.shape)[1:] == [2048, 1, 1]:
                x = x.reshape([-1, 2048])

            x = self.layer_operations[l](x)

            if l in layers:
                outputs[l + 1] = self.linear_layers[l + 1](x.reshape([x.shape[0], -1]))
        return outputs


free_mem()
resnet152_wrapper = WrapModelForResNet152(model.imported_model, make_multichannel_input, classes=classes)
resnet152_wrapper.multichannel_fn = make_multichannel_input
resnet152_wrapper = resnet152_wrapper.to("cuda")
assert all([resnet152_wrapper.predict_from_layer(torch.Tensor(np.zeros((2, 3, 32, 32))).cuda(), layer_i).shape == torch.Size([2, classes]) for layer_i in range(53)])  # shape is [2, num_classes]


class LinearNet(nn.Module):  # just for a single intermediate layer
    def __init__(self, model, layer_i):
        super(LinearNet, self).__init__()
        self.model = model
        self.layer_i = layer_i

    def forward(self, inputs):
        return self.model.predict_from_layer(inputs, self.layer_i)


free_mem()
backbone_model = resnet152_wrapper.cpu()
backbone_model = copy.deepcopy(backbone_model)
backbone_model = backbone_model.to("cuda")
backbone_model.eval()
linear_model = LinearNet(backbone_model, 5).to("cuda")  # just to have it ready

linear_layers_collected_dict = dict()
layers_to_use = [20, 30, 35, 40, 45, 50, 52]  # just a subset - super early ones are bad on anything harder than CIFAR-10
for layer_i in reversed(layers_to_use):
    print(f"------- training layer={layer_i}")

    train_accs = []
    test_accs = []

    free_mem()
    linear_model.layer_i = layer_i
    linear_model.fixed_mode = "train"
    with torch.autocast("cuda"):
        linear_model = train_model(
            model_in=linear_model,
            images_in=images_train_np[:],
            labels_in=labels_train_np[:],
            epochs=1,
            lr=3.3e-5,  # not optimal
            batch_size=32,  # reduce on mem issues
            optimizer_in=optim.Adam,
            subset_only=linear_model.model.linear_layers[layer_i].parameters(),  # just the linear projection
            mode="train",
            use_adversarial_training=False,
            adversarial_epsilon=None,
            skip_test_set_eval=False,
        )
    linear_layers_collected_dict[layer_i] = copy.deepcopy(backbone_model.linear_layers[layer_i])

# load everything to a single model
for layer_i in layers_to_use:
    print(f"training layer {layer_i} in {layers_to_use}")
    backbone_model.linear_layers[layer_i] = copy.deepcopy(linear_layers_collected_dict[layer_i])
backbone_model.cuda()
backbone_model.eval()

# save the model
save_file(backbone_model, weights_path / "original_model.safetensors")

In [ ]:
#
# benchmarking
#

test_acc_by_layer = []
for layer_i in layers_to_use:
    linear_model = LinearNet(backbone_model, layer_i).to("cuda")
    linear_model.eval()
    test_hits, test_count, _ = eval_model(linear_model, images_test_np, labels_test_np)
    test_acc_by_layer.append(test_hits / test_count)
    print(f"layer={layer_i} test={test_hits}/{test_count}={test_hits/test_count}")

plt.figure(figsize=(7, 5), dpi=100)
plt.title("Accuracy at intermediate layers", fontsize=14)
plt.plot(layers_to_use, test_acc_by_layer, marker="o", color="navy", label="Test")
plt.legend(fontsize=16)
plt.xlabel("Layer", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()